<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/ph-waterbodies?scriptVersionId=163647836" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## <center>How to monitor pH in water bodies using Remote sensing techniques in Google Earth Engine</center>
---
reference: [https://www.youtube.com/watch?v=llLejFq0hN0](https://www.youtube.com/watch?v=llLejFq0hN0)

In [1]:
!pip install -q geemap

In [67]:
import geemap.foliumap as geemap
import geemap.colormaps as cmap
import ee

In [104]:
ee.Authenticate()
ee.Initialize()

Applications to global departments

In [107]:
Map = geemap.Map()

department_name = "Ancash"

department = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq("ADM1_NAME", department_name))

img = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
                                    .filterBounds(department)\
                                    .filterDate("2022-01-01", "2022-12-31")\
                                    .filterMetadata("CLOUD_COVER_LAND", "less_than", 20)\
                                    .median()\
                                    .clip(department)

img_params = {
    'bands': ['B6', 'B5', 'B2'],
    'min' : 0.0,
    'max' : 0.4
}

def MNDW_f(img):
    MNDWI = img.normalizedDifference(["B3", "B5"]).rename("MNDWI")
    return img.addBands(MNDWI).updateMask(MNDWI.gt(0))

img_water = MNDW_f(img)

def pH_f(img):
    pH = img.expression("9.9977 + (-110.1097 * B1) + (17.4231 * B3) + (49.6782 * B5)", {"B1": img.select("B1"),
                                                                                       "B3": img.select("B3"),
                                                                                       "B5": img.select("B5")}).rename("pH")
    return img.addBands(pH)

img_pH = pH_f(img_water)

viz = {"min": -4.96, "max": 3.95, "palette": cmap.palettes.jet}

Map.centerObject(department)

Map.addLayer(img, img_params, "image")
Map.addLayer(img_water.select("MNDWI"), {"palette": "cyan"}, "img water")
Map.addLayer(img_pH.select("pH"), viz, "img ph")
Map.addLayer(ee.Image().paint(department, 0, 2.5), {'palette': 'red'}, f"departament {department_name.lower()}")

Map.add_text(f"Ph map of the water bodies of the department of {department_name}", 
             position = "bottomleft", 
             bg_color = None, 
             bold = True,
            fontsize = 15)

Map